# Deployable Payload Flight Simulation Example

Here we try to demonstrate how to use RocketPy to simulate a flight of a rocket
that presents a deployable payload.

To run this notebook, we will need:

*   RocketPy
*   netCDF4 (to get weather forecasts)
*   Data files (we will clone RocketPy's repository for these)

Therefore, let's run the following lines of code:

In [ ]:
!pip install rocketpy netCDF4
!git clone https://github.com/RocketPy-Team/RocketPy.git

In [ ]:
import os

os.chdir("RocketPy/docs/notebooks")

Now we can start!

Here we go through a simplified rocket trajectory simulation to get you started. Let's start by importing the rocketpy module.

In [ ]:
from rocketpy import Environment, SolidMotor, Rocket, Flight, Function, utilities

If you are using a version of Jupyter Notebooks, it is recommended to run the following lines of code to make plots that will be shown later interactive and/or higher quality.

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

## Setting Up a Simulation

### Creating an Environment for Spaceport America

In [ ]:
Env = Environment(
    rail_length=5.2, latitude=32.990254, longitude=-106.974998, elevation=1400
)

To get weather data from the GFS forecast, available online, we run the following lines.
See [Environment Class Usage](environment_class_usage.ipynb) for more information on how to use the Environment class.

In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

Env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))  # Hour given in UTC time

Env.set_atmospheric_model(type="Forecast", file="GFS")
Env.max_expected_height = 8000

In [ ]:
Env.info()

### Creating a Motor

A solid rocket motor is used in this case. See [Solid Motor Class Usage](solid_motor_class_usage.ipynb) for more information on how to use the Motor class.

In [ ]:
Pro75M1670 = SolidMotor(
    thrust_source="../../data/motors/Cesaroni_M1670.eng",
    burn_out=3.9,
    grain_number=5,
    grain_separation=5 / 1000,
    grain_density=1815,
    grain_outer_radius=33 / 1000,
    grain_initial_inner_radius=15 / 1000,
    grain_initial_height=120 / 1000,
    nozzle_radius=33 / 1000,
    throat_radius=11 / 1000,
    grains_center_of_mass_position=-0.85704,
    interpolation_method="linear",
)

In [ ]:
Pro75M1670.info()

## Simulating the First Flight Stage

Let's start to simulate our rocket's flight. We will use the Environment and Motor objects we created before.

We will assume that the payload is ejected at apogee, however, this can be modified if needed.

We start by defining the value of each relevant mass, ensuring they are correct before continuing.

In [ ]:
# 16.241 is the mass of the rocket including the payload but without the propellant
payload_mass = 4.5  # in kg
rocket_mass = 16.241 - payload_mass  # in kg

print("Rocket dry mass: {:.4} kg (with Payload)".format(rocket_mass + payload_mass))
print("Propellant Mass: {:.4} kg".format(Pro75M1670.mass(0)))
print("Payload Mass: {:.4} kg".format(payload_mass))
print(
    "Fully loaded Rocket Mass: {:.4} kg".format(
        rocket_mass + Pro75M1670.mass(0) + payload_mass
    )
)

Then we define our rocket. See [Getting Started](getting_started_colab.ipynb) for more information on defining a Rocket.

In [ ]:
Rocket1 = Rocket(
    radius=127 / 2000,
    mass=rocket_mass + payload_mass,
    inertia_i=6.60,
    inertia_z=0.0351,
    power_off_drag="../../data/calisto/powerOffDragCurve.csv",
    power_on_drag="../../data/calisto/powerOnDragCurve.csv",
    center_of_dry_mass_position=0,
    coordinate_system_orientation="tail_to_nose",
)

Rocket1.add_motor(motor=Pro75M1670, position=-1.255)

Rocket1.set_rail_buttons([0.2, -0.5])

nosecone_rocket1 = Rocket1.add_nose(
    length=0.55829, kind="vonKarman", position=1.278
)

finset_rocket1 = Rocket1.add_fins(
    4, span=0.100, root_chord=0.120, tip_chord=0.040, position=-1.04956
)

tail_rocket1 = Rocket1.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)

In [ ]:
Rocket1.info()

Finally we create the flight simulation of this rocket, stopping at apogee

In [ ]:
RocketFlight1 = Flight(
    rocket=Rocket1, environment=Env, inclination=85, heading=25, terminate_on_apogee=True, name="RocketFlight1"
)

## Start the Second Flight Stage

Now we will simulate the second flight stage, which is the landing phase of our Rocket.
Here we will consider that the payload was ejected at the apogee of the first stage.
Therefore we should be careful with the value of its mass.

In [ ]:
Rocket2 = Rocket(
    radius=127 / 2000,
    mass=rocket_mass,   # Rocket with the eject payload
    inertia_i=6.60,
    inertia_z=0.0351,
    power_off_drag=1,
    power_on_drag=1,
    center_of_dry_mass_position=0,
    coordinate_system_orientation="tail_to_nose",
)



def drogue_trigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False


def main_trigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate main when vz < 0 m/s and z < 800 + 1400 m (+1400 due to surface elevation).
    return True if y[5] < 0 and y[2] < 800 + 1400 else False


# Define Parachutes for the rocket
Main_Rocket2 = Rocket2.add_parachute(
    "Main",
    CdS=7.2,
    trigger=main_trigger,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Drogue_Rocket2 = Rocket2.add_parachute(
    "Drogue",
    CdS=0.72,
    trigger=drogue_trigger,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

In [ ]:
Rocket2.info()

The magic line `initialSolution=RocketFlight1` will make the simulation start from the end of the first stage. 

This will simulate our rocket with its payload ejected, after reaching apogee.

In [ ]:
RocketFlight2 = Flight(
    rocket=Rocket2,
    environment=Env,
    inclination=0,
    heading=0,
    max_time=600,
    initial_solution=RocketFlight1,
    name="RocketFlight2",
)

## Simulating the Third Flight Stage - Payload Flight

Here we will simulate the payload flight, which is the third flight stage of our Rocket.
The Payload will be ejected at the apogee of the first stage.
Here, it will be modeled as a "dummy" rocket, which does not have any aerodynamic surfaces to stabilize it, nor a motor that ignites.
It does, however, have parachutes.

In [ ]:
# Define the "Payload Rocket"

PayloadRocket = Rocket(
    radius=127 / 2000,
    mass=payload_mass,
    # The next arguments do not matter for the simulation since it is only a dummy rocket
    # so we just add symbolic values:
    inertia_i=6.60,
    inertia_z=0.0351,
    power_off_drag=0.5,
    power_on_drag=0.5,
    center_of_dry_mass_position=0,
    coordinate_system_orientation="tail_to_nose",
)


def drogue_trigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False


def main_trigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate main when vz < 0 m/s and z < 800 + 1400 m (+1400 due to surface elevation).
    return True if y[5] < 0 and y[2] < 800 + 1400 else False


PayloadDrogue = PayloadRocket.add_parachute(
    "Drogue",
    CdS=0.35,
    trigger=drogue_trigger,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

PayloadMain = PayloadRocket.add_parachute(
    "Main",
    CdS=4.0,
    trigger=main_trigger,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

The magic line `initialSolution=RocketFlight1` will make the simulation start from the end of the first stage.

In [ ]:
PayloadFlight = Flight(
    rocket=PayloadRocket,
    environment=Env,
    inclination=0,
    heading=0,
    max_time=600,
    initial_solution=RocketFlight1,
    name="PayloadFlight",
)

## Plotting Everything together

We will invoke a method from RocketPy's utilities class in order to visualize 
the trajectory.

In [ ]:
from rocketpy.plots.compare import CompareFlights

Then we create the `comparison` object, an instance of CompareFligths class

In [ ]:
comparison = CompareFlights([RocketFlight1, RocketFlight2, PayloadFlight])

And, finally, we are able to plot different aspects of the comparison object.

In [ ]:
comparison.trajectories_3d(legend=True)

In [ ]:
comparison.positions()

In [ ]:
comparison.velocities()

In [ ]:
comparison.accelerations()

In [ ]:
comparison.aerodynamic_forces()

In [ ]:
comparison.aerodynamic_moments()

In [ ]:
comparison.angles_of_attack()